<a href="https://colab.research.google.com/github/Niknik26/spark_task/blob/main/Spark_tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Самостоятельное задание  №1. Погода

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=7b8bf4b646d11ee3aee7f9d4229209da5cb7ba313d6e3307804b4f999af96849
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [7]:
# Самостоятельное задание  №1. Погода
# Создание SparkSession
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, mean, max, avg, month
import pandas as pd

spark = SparkSession.builder.appName("Read weather").getOrCreate()

# Чтение CSV-файла
#df = spark.read.csv("/content/sample_data/weather_data_n.csv", header=True, inferSchema=True)
df = spark.read.csv("/content/weather_data.csv", header=True, inferSchema=True)

# Cр. значения по метеостанциям
# mean_t = df.select(mean(col('temperature'))).collect()[0][0]
# df_t = df.na.fill({"temperature": mean_t}).show()
# mean_p = df.select(mean(col('precipitation'))).collect()[0][0]
# df_p = df.na.fill({"precipitation": mean_p}).show()
# mean_w = df.select(mean(col('wind_speed'))).collect()[0][0]
# df_w = df.na.fill({"wind_speed": mean_w}).show()

# Вывод строк с пустыми данными, если есть.
df_nulls = df.filter(col("temperature").isNull()|col("precipitation").isNull()|col("wind_speed").isNull())

# Подсчет количества таких строк
null_count = df_nulls.count()
df_nulls.show()
if null_count > 0:
    print(f"Отсутствуют значения погоды. Всего строк {null_count}")
else:
    print("НЕТ пропущенных значений")

# Cредние значения по метеостанциям
mean_all = {
    "temperature": df.select(mean(col("temperature"))).collect()[0][0],
    "precipitation": df.select(mean(col("precipitation"))).collect()[0][0],
    "wind_speed": df.select(mean(col("wind_speed"))).collect()[0][0]
}
# Заполнить отсутствующие значения
df_filled = df.na.fill(mean_all)

# 3. Анализ данных:
# 3.1.Найдите топ-5 самых жарких дней за все время наблюдений.
# df.select(max("temperature")).show()
df.select(col("date"),col("temperature")).sort(col("temperature").desc(),col("date")).limit(5).show()

# 3.2.Найдите метеостанцию с наибольшим количеством осадков за последний год.
dfGroup = df.filter(col("date") >= "2023-01-01").select(col("station_id"),col("precipitation")).groupBy("station_id").sum().sort(col("sum(precipitation)").desc())
dfGroup.limit(1).show()

# 3.3.Подсчитайте среднюю температуру по месяцам за все время наблюдений.
# Месяц даты извлекаем
df = df.withColumn("month", month(col("date")))

df_grouped = df.groupBy("month").mean("precipitation").show()

spark.stop()

+----------+----+-----------+-------------+----------+
|station_id|date|temperature|precipitation|wind_speed|
+----------+----+-----------+-------------+----------+
+----------+----+-----------+-------------+----------+

НЕТ пропущенных значений
+----------+------------------+
|      date|       temperature|
+----------+------------------+
|2021-08-20|39.982828249354846|
|2023-12-02| 39.96797489293784|
|2022-03-28|  39.8246894248997|
|2019-02-11| 39.76737697836647|
|2020-06-10| 39.69147838355929|
+----------+------------------+

+----------+------------------+
|station_id|sum(precipitation)|
+----------+------------------+
| station_5| 642.9302626767898|
+----------+------------------+

+-----+------------------+
|month|avg(precipitation)|
+-----+------------------+
|   12| 30.01257487211919|
|    1|25.788403692179923|
|    6| 25.48369925567644|
|    3|25.992138694271425|
|    5|24.771692387500345|
|    9|26.306950943105992|
|    4| 24.20807094379405|
|    8|27.466365894503493|
|    7|

Самостоятельное задание  №2. Книги и авторы

In [ ]:
# Самостоятельное задание  №2. Книги и авторы
# Создание SparkSession
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, mean, max, avg, month, year, when
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("BOOKS").getOrCreate()

# Чтение CSV-файла
df_books = spark.read.csv("/content/books.csv", header=True, inferSchema=True)
df_authors = spark.read.csv("/content/authors.csv", header=True, inferSchema=True)

# Преобразование publish_date и birth_date в формат даты.
df_b = df_books.withColumn("publish_date", to_date(col("publish_date"), "yyyy-MM-dd"))
df_a = df_authors.withColumn("birth_date", to_date(col("birth_date"), "yyyy-MM-dd"))

# 4.1 топ-5 авторов, книги которых принесли наибольшую выручку
# Объедините таблицы books и authors по author_id.
inner_join_df = df_a.join(df_b, df_a.author_id == df_b.author_id, "inner").drop(df_b["author_id"])
inner_join_df.show()

# Группировка по author_id из df_a и вычисление суммы price из df_b, переименование столбца(sum(price) в total_revenue)упорядочение по убыванию и выбор топ 5 авторов
aa = inner_join_df.groupBy("author_id").sum("price")
aa = aa.withColumnRenamed("sum(price)", "total_revenue")
aa1 = aa.sort(col("total_revenue").desc()).limit(5)

# Присоединение столбца "name" и удаление лишних столбцов
inner_aa= aa1.join(df_a, aa1.author_id == df_a.author_id, "left_outer").drop(aa1["author_id"]).select("author_id", "name", "total_revenue")
inner_aa.show()

# 4.2 количество книг в каждом жанре.
sum_g=inner_join_df.groupBy("genre").count().sort(col("count").desc())
sum_g.show()

# 4.3 средняя цену книг по каждому автору   удалил .desc()
avg_price = inner_join_df.groupBy("author_id").mean("price").sort(col("avg(price)"))
avg_price = avg_price.withColumnRenamed("avg(price)", "average_price")

# Присоединение столбца "name" и удаление лишних столбцов
avg_price = avg_price.join(df_a, avg_price.author_id == df_a.author_id, "left_outer").drop(avg_price["author_id"]).select("author_id", "name", "average_price")
avg_price = avg_price.sort(col("average_price").desc())
avg_price.show()

# 4.4 Книги, опубликованные после 2000 года, и отсортируйте их по цене
year = inner_join_df.filter(year("publish_date") >= 2000)
book_year = year.select("author_id", "book_id", "title", "genre", "price", "publish_date", "name", "birth_date", "country").sort(col("price").desc())
book_year.show()

spark.stop()



+---------+---------+----------+---------+-------+-------+-----------+-----+------------+
|author_id|     name|birth_date|  country|book_id|  title|      genre|price|publish_date|
+---------+---------+----------+---------+-------+-------+-----------+-----+------------+
|        2| Author_2|1965-12-31|   Canada|      1| Book_1|    Mystery|73.57|  1980-12-31|
|        1| Author_1|1960-12-31|    India|      2| Book_2|Non-Fiction| 41.1|  1982-12-31|
|       10|Author_10|2005-12-31|    India|      3| Book_3|    Fiction|10.63|  1984-12-31|
|        9| Author_9|2000-12-31|Australia|      4| Book_4|Non-Fiction|46.31|  1986-12-31|
|        7| Author_7|1990-12-31|      USA|      5| Book_5|    Science|31.13|  1988-12-31|
|        4| Author_4|1975-12-31|       UK|      6| Book_6|Non-Fiction| 83.7|  1990-12-31|
|        6| Author_6|1985-12-31|      USA|      7| Book_7|Non-Fiction|40.36|  1992-12-31|
|        2| Author_2|1965-12-31|   Canada|      8| Book_8|Non-Fiction|84.48|  1994-12-31|
|        7

Самостоятельное задание  №3. Фильмы и актеры. С использованием временных таблиц и SQL

In [120]:
#Самостоятельное задание  №3. Фильмы и актеры. С использованием временных таблиц и SQL
# Создание SparkSession
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, mean, max, avg, month, year, when
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("Movies").getOrCreate()

# Чтение CSV-файла
df_movie_actors = spark.read.csv("/content/movie_actors.csv", header=True, inferSchema=True)
df_movie = spark.read.csv("/content/movies.csv", header=True, inferSchema=True)
df_actors = spark.read.csv("/content/actors.csv", header=True, inferSchema=True)

# Регистрация DataFrame как временные таблицы
df_movie_actors.createOrReplaceTempView("movie_actorsT")
df_movie.createOrReplaceTempView("movieT")
df_actors.createOrReplaceTempView("actorsT")

# 4.1 топ-5 жанров по количеству фильмов.
genre_name = spark.sql("""
SELECT genre, COUNT(movie_id) AS num_movies
FROM movieT
GROUP BY genre
ORDER BY num_movies DESC
""")
genre_name.show()

# 4.2 актер с наибольшим количеством фильмов.
# Выполнение JOIN-запроса три таблицы с использованием SQL, без дублирующих связывающих столбцов
join_df = spark.sql("""
SELECT mt.movie_id, title, genre, release_date, budget, at.actor_id, name, birth_date, country
FROM movieT mt
JOIN movie_actorsT mat
ON mt.movie_id = mat.movie_id
JOIN actorsT at
ON mat.actor_id = at.actor_id
""")
join_df.show()
# Регистрация join_df как временной таблицы
join_df.createOrReplaceTempView("join_df_temp")

act_max = spark.sql("""
SELECT name, COUNT (movie_id) AS num_movies
FROM join_df_temp
GROUP BY name
ORDER BY num_movies DESC
limit(1)
""")
act_max.show()

# 4.3 Cредний бюджет фильмов по жанрам
bud = spark.sql("""
SELECT genre, AVG(budget) AS avg_budget
FROM movieT
GROUP BY genre
""")
bud.show()

# 4.4 Фильмы, в которых снялось более одного актера из одной страны.
more1act = spark.sql("""
SELECT title, country, COUNT(DISTINCT actor_id) AS num_actors
FROM join_df_temp
GROUP BY title, country
HAVING num_actors > 1
""")
more1act.show()


+------+----------+
| genre|num_movies|
+------+----------+
| Drama|         6|
|Action|         6|
|Comedy|         4|
|Horror|         2|
|Sci-Fi|         2|
+------+----------+

+--------+-------+------+------------+-------------+--------+--------+----------+---------+
|movie_id|  title| genre|release_date|       budget|actor_id|    name|birth_date|  country|
+--------+-------+------+------------+-------------+--------+--------+----------+---------+
|       1|Movie_1|Horror|  2000-12-31|8.660058311E7|      13|Actor_13|1984-12-31|       UK|
|       1|Movie_1|Horror|  2000-12-31|8.660058311E7|      16|Actor_16|1990-12-31|    India|
|       1|Movie_1|Horror|  2000-12-31|8.660058311E7|      25|Actor_25|2008-12-31|    India|
|       1|Movie_1|Horror|  2000-12-31|8.660058311E7|       8| Actor_8|1974-12-31|Australia|
|       1|Movie_1|Horror|  2000-12-31|8.660058311E7|      25|Actor_25|2008-12-31|    India|
|       2|Movie_2|Comedy|  2001-12-31|1.274740083E7|      17|Actor_17|1992-12-31|  